🧑‍💼 Mini Real-Life Project: Motion Detection in Surveillance Footage using Frame Differencing and Gaussian Smoothing

**✅ Why this matters**

In security cameras, detecting motion accurately is critical. But raw footage often contains noise due to lighting changes, compression artifacts, or hardware limitations.

We'll combine:

Frame differencing for detecting motion

Gaussian filtering to suppress noise

Averaging and WMA to clean up unstable pixel shifts



**🎯 Project Goal**

Detect movement between two consecutive video frames while reducing false positives caused by noise.

In [3]:
import cv2
import numpy as np
import gradio as gr

ROI_X, ROI_Y, ROI_W, ROI_H = 100, 100, 300, 300
prev_frame = None

def get_combined_kernel():
    wma_kernel = np.array([[1, 2, 1],
                           [2, 4, 2],
                           [1, 2, 1]], dtype=np.float32)
    wma_kernel /= wma_kernel.sum()
    return wma_kernel

def process_image_mode(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply ROI
    roi = gray[ROI_Y:ROI_Y+ROI_H, ROI_X:ROI_X+ROI_W]

    # Apply Gaussian and WMA filters
    blurred = cv2.GaussianBlur(roi, (5, 5), 1.5)
    kernel = get_combined_kernel()
    filtered = cv2.filter2D(blurred, -1, kernel)

    # Replace ROI in the original image
    output = image.copy()
    output[ROI_Y:ROI_Y+ROI_H, ROI_X:ROI_X+ROI_W] = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)

    # Draw ROI rectangle
    cv2.rectangle(output, (ROI_X, ROI_Y), (ROI_X+ROI_W, ROI_Y+ROI_H), (0, 255, 0), 2)
    return output

# For real-time video capture
def process_video_frame():
    global prev_frame
    cap = cv2.VideoCapture(0)
    success, frame = cap.read()
    if not success:
        cap.release()
        return None
    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = gray[ROI_Y:ROI_Y+ROI_H, ROI_X:ROI_X+ROI_W]
    blurred = cv2.GaussianBlur(roi, (5, 5), 1.5)
    kernel = get_combined_kernel()
    filtered = cv2.filter2D(blurred, -1, kernel)

    motion_mask = np.zeros_like(roi)
    if prev_frame is not None:
        diff = cv2.absdiff(prev_frame, filtered)
        _, motion_mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    prev_frame = filtered.copy()

    color_mask = cv2.cvtColor(motion_mask, cv2.COLOR_GRAY2BGR)
    frame[ROI_Y:ROI_Y+ROI_H, ROI_X:ROI_X+ROI_W] = cv2.addWeighted(
        frame[ROI_Y:ROI_Y+ROI_H, ROI_X:ROI_X+ROI_W], 0.6, color_mask, 0.4, 0)
    cv2.rectangle(frame, (ROI_X, ROI_Y), (ROI_X+ROI_W, ROI_Y+ROI_H), (0, 255, 0), 2)
    cap.release()
    return frame

def interface_choice(choice, image_input):
    if choice == "Upload Image":
        return process_image_mode(image_input)
    elif choice == "Live Camera":
        return process_video_frame()

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🎯 Real-Life Motion Detection App with ROI Filtering")

    mode = gr.Radio(["Upload Image", "Live Camera"], label="Choose Mode", value="Upload Image")
    image_input = gr.Image(type="numpy", label="Upload an Image", visible=True)
    output_image = gr.Image(type="numpy", label="Processed Output")

    run_button = gr.Button("Run Detection")

    def toggle_visibility(mode_choice):
        return gr.update(visible=(mode_choice == "Upload Image"))

    mode.change(fn=toggle_visibility, inputs=mode, outputs=image_input)
    run_button.click(fn=interface_choice, inputs=[mode, image_input], outputs=output_image)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bafa3b16fa4bd4c6ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
